Questions answering with embeddings.
Adapted from [OpenAI Notebook](https://github.com/openai/openai-cookbook/blob/main/examples/Question_answering_using_embeddings.ipynb).

In [1]:
from minichain import TemplatePrompt, Prompt, start_chain, show_log
import datasets
import numpy as np

Load data with embeddings (computed beforehand)

In [2]:
olympics = datasets.load_from_disk("olympics.data")
olympics.add_faiss_index("embeddings")

  0%|          | 0/4 [00:00<?, ?it/s]

Dataset({
    features: ['title', 'heading', 'content', 'tokens', 'embeddings'],
    num_rows: 3964
})

Fast KNN retieval prompt 

In [3]:
class KNNPrompt(Prompt):
    def parse(self, out, inp):
        res = olympics.get_nearest_examples("embeddings",
                                            np.array(out), 3)
        return {"question": inp, "docs": res.examples["content"]}

QA prompt to ask question with examples

In [4]:
class QAPrompt(TemplatePrompt):
    template_file = "qa.pmpt.tpl"

In [5]:
with start_chain("qa") as backend:
    question = "Who won the 2020 Summer Olympics men's high jump?"
    prompt =  KNNPrompt(backend.OpenAIEmbed()).chain(QAPrompt(backend.OpenAI()))
    result = prompt(question)
    print(result)

Italian athlete Gianmarco Tamberi and Qatari athlete Mutaz Essa Barshim emerged as joint winners of the event following a tie between both of them as they cleared 2.37m. Both Tamberi and Barshim agreed to share the gold medal.


In [6]:
QAPrompt().show({"question": "Who won the race?", "docs": ["doc1", "doc2", "doc3"]},
                "Joe Bob")

HTML(html='\n<!-- <link rel="stylesheet" href="https://cdn.rawgit.com/Chalarangelo/mini.css/v3.0.1/dist/mini-default.min.css"> -->\n <main class="container">\n\n<h3>QAPrompt</h3>\n\n<dl>\n  <dt>Input:</dt>\n  <dd>\n<div class="highlight"><pre><span></span><span class="p">{</span><span class="s1">&#39;question&#39;</span><span class="p">:</span> <span class="s1">&#39;Who won the race?&#39;</span><span class="p">,</span> <span class="s1">&#39;docs&#39;</span><span class="p">:</span> <span class="p">[</span><span class="s1">&#39;doc1&#39;</span><span class="p">,</span> <span class="s1">&#39;doc2&#39;</span><span class="p">,</span> <span class="s1">&#39;doc3&#39;</span><span class="p">]}</span>\n</pre></div>\n\n\n  </dd>\n\n  <dt> Full Prompt: </dt>\n  <dd>\n    <details>\n      <summary>Prompt</summary>\n      <p>Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text below, say "I don\'t know."<br><br>Context:<br><br><br>* doc1<br><br>* doc2<br><br>* doc3<br><br><br> Q: <div style=\'color:red\'>Who won the race?</div><br><br> A: <br></p>\n    </details>\n  </dd>\n\n  <dt> Response: </dt>\n  <dd>\n    Joe Bob\n  </dd>\n\n  <dt>Value:</dt>\n  <dd>\n<div class="highlight"><pre><span></span><span class="n">Joe</span> <span class="n">Bob</span>\n</pre></div>\n\n  </dd>\n</main>\n')

In [7]:
show_log("qa.log")

d70adf77-3e40-4477-b46d-6dca41a55650
└── <class '__main__.KNNPrompt'>/1 ⇒ started 2023-02-26 01:41:04Z ⧖ 0.474s
    ├── Input Function/2/1 ⇒ started 2023-02-26 01:41:04Z ⧖ 0.000s
    │   ├── input: Who won the 2020 Summer Olympics men's high jump?
    │   └── Input Function/2/2 ⇒ succeeded 2023-02-26 01:41:04Z
    ├── Prompted/3/1 ⇒ started 2023-02-26 01:41:04Z ⧖ 0.467s
    │   ├── prompt: Who won the 2020 Summer Olympics men's high jump?
    │   └── Prompted/3/2 ⇒ succeeded 2023-02-26 01:41:05Z
    ├── Result/4/1 ⇒ started 2023-02-26 01:41:05Z ⧖ 0.006s
    │   ├── result: 
    │   │   ├── 0: 0.01102200336754322
    │   │   ├── 1: -0.003180644242092967
    │   │   ├── 2: 0.014023388735949993
    │   │   ├── 3: -0.022208986803889275
    │   │   ├── 4: 0.007785836234688759
    │   │   ├── 5: 0.02764067053794861
    │   │   ├── 6: -0.020495722070336342
    │   │   ├── 7: -0.02677769400179386
    │   │   ├── 8: -0.01150425523519516
    │   │   ├── 9: -0.005545901134610176
    │   │   ├── 1